In [1]:
from rule_based_segmentation import mask_non_skin
from ita_extraction import ITA, empirical_classification, kinyananjui_classification
import os
import pandas as pd
import imageio as im
import time

In [2]:
def func(file_name,real_class):
    image = im.imread(file_name)
    direct_ita = ITA(image)
    skin,__ = mask_non_skin(image)
    segmented_ita = ITA(skin)
    
    #kinyananjui_class_direct_ita = kinyananjui_classification(direct_ita)
    #empirical_class_direct_ita = empirical_classification(direct_ita)
    kinyananjui_class_segmented_ita = kinyananjui_classification(segmented_ita)
    empirical_class_segmented_ita = empirical_classification(segmented_ita)
    
    #direct_kinyananjui_hit = 1 if kinyananjui_class_direct_ita == real_class else 0
    #direct_empirical_hit = 1 if empirical_class_direct_ita == real_class else 0
    kinyananjui_hit = 1 if kinyananjui_class_segmented_ita == real_class else 0
    empirical_hit = 1 if empirical_class_segmented_ita == real_class else 0
    
    
    #direct_kinyananjui_relaxed_hit = 1 if kinyananjui_class_direct_ita >= real_class-1 and kinyananjui_class_direct_ita <= real_class+1 else 0
    #direct_empirical_relaxed_hit = 1 if empirical_class_direct_ita >= real_class-1 and empirical_class_direct_ita <= real_class+1 else 0
    kinyananjui_relaxed_hit = 1 if kinyananjui_class_segmented_ita >= real_class-1 and kinyananjui_class_segmented_ita <= real_class+1 else 0
    empirical_relaxed_hit = 1 if empirical_class_segmented_ita >= real_class-1 and empirical_class_segmented_ita <= real_class+1 else 0
    
    return segmented_ita,direct_ita,kinyananjui_class_segmented_ita,empirical_class_segmented_ita,kinyananjui_hit,empirical_hit,kinyananjui_relaxed_hit,empirical_relaxed_hit,real_class

In [3]:
df_test = pd.read_csv('../datasets/fitzpatrick17k/fitzpatrick17k_train.csv')

df_test

,md5hash,fitzpatrick,label,nine_partition_label,three_partition_label,url,url_alphanum
0,dae2d6176ea28b4d991c573c8320b0cf,4,lichen planus,inflammatory,non-neoplastic,http://atlasdermatologico.com.br/img?imageId=3739,httpwwwatlasdermatologicocombrimgimageId3739.jpg
1,dcba3efe0fe28be2b0032df9470c320d,5,behcets disease,inflammatory,non-neoplastic,http://atlasdermatologico.com.br/img?imageId=8633,httpwwwatlasdermatologicocombrimgimageId8633.jpg
2,9a9ded2de5e2c20a90cdfc2971e2e155,2,pityriasis lichenoides chronica,inflammatory,non-neoplastic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicppityr...
3,d6166c9947e7e99915ff4d6907ddec44,2,mycosis fungoides,malignant cutaneous lymphoma,malignant,http://atlasdermatologico.com.br/img?imageId=4594,httpwwwatlasdermatologicocombrimgimageId4594.jpg
4,8ea58508e89d61a98320acb03139b8bb,3,urticaria pigmentosa,inflammatory,non-neoplastic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicuurtic...
...,...,...,...,...,...,...,...
14867,2b7d639b26abd635dd0d312969991646,-1,psoriasis,inflammatory,non-neoplastic,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicppsori...
14868,3e96f0e6bdf82fe4ee5d7635c76f32a7,1,nevus sebaceous of jadassohn,benign epidermal,benign,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicnnevus...
14869,f28a23e1108a79279117c9c6c3776e66,2,syringoma,benign dermal,benign,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicssyrin...
14870,c362bf63c2203189f622a4de29e5c844,2,sarcoidosis,inflammatory,non-neoplastic,http://atlasdermatologico.com.br/img?imageId=6528,httpwwwatlasdermatologicocombrimgimageId6528.jpg


In [4]:
image_dir = '../datasets/fitzpatrick17k/images'

In [5]:
df_result = pd.DataFrame()
df_result

""


In [6]:
start = time.time()
result = df_test.apply(lambda row: func(os.path.join(image_dir,row['md5hash']+'.jpg'),row['fitzpatrick']),axis=1)
stop = time.time()
stop-start

/opt/conda/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/store/vrusso/notebooks/ita_extraction.py:39: RuntimeWarning: Mean of empty slice
  std, mean = np.nanstd(L), np.nanmean(L)
/store/vrusso/notebooks/ita_extraction.py:44: RuntimeWarning: Mean of empty slice
  std, mean = np.nanstd(B), np.nanmean(B)
/store/vrusso/notebooks/ita_extraction.py:52: RuntimeWarning: Mean of empty slice
  ITA = math.atan2(np.nanmean(L) - 50, np.nanmean(B)) * (180 / np.pi)


4051.453592777252

In [7]:
df_aux = pd.DataFrame(list(result))
df_aux

,0,1,2,3,4,5,6,7,8
0,-10.052935,-6.445718,6,5,0,0,0,1,4
1,-25.748827,-71.203786,6,6,0,0,1,1,5
2,45.145774,48.991663,2,1,1,0,1,1,2
3,44.147618,-97.912738,2,1,1,0,1,1,2
4,29.865609,9.164579,3,2,1,0,1,1,3
...,...,...,...,...,...,...,...,...,...
14867,16.171660,10.484340,5,3,0,0,0,0,-1
14868,51.907781,52.943428,2,1,0,1,1,1,1
14869,63.403899,58.011177,1,1,0,0,1,1,2
14870,27.617172,2.371818,4,2,0,1,0,1,2


In [8]:
df_result['segmented_ita'] = df_aux[0]
df_result['direct_ita'] = df_aux[1]
df_result['kinyananjui_class'] = df_aux[2]
df_result['empirical_class'] = df_aux[3]
df_result['kinyananjui_hit'] = df_aux[4]
df_result['empirical_hit'] = df_aux[5]
df_result['kinyananjui_relaxed_hit'] = df_aux[6]
df_result['empirical_relaxed_hit'] = df_aux[7]
df_result['real_class'] = df_aux[8]



df_result

,segmented_ita,direct_ita,kinyananjui_class,empirical_class,kinyananjui_hit,empirical_hit,kinyananjui_relaxed_hit,empirical_relaxed_hit,real_class
0,-10.052935,-6.445718,6,5,0,0,0,1,4
1,-25.748827,-71.203786,6,6,0,0,1,1,5
2,45.145774,48.991663,2,1,1,0,1,1,2
3,44.147618,-97.912738,2,1,1,0,1,1,2
4,29.865609,9.164579,3,2,1,0,1,1,3
...,...,...,...,...,...,...,...,...,...
14867,16.171660,10.484340,5,3,0,0,0,0,-1
14868,51.907781,52.943428,2,1,0,1,1,1,1
14869,63.403899,58.011177,1,1,0,0,1,1,2
14870,27.617172,2.371818,4,2,0,1,0,1,2


In [9]:
df_result.to_csv('baseline_train_results.csv',index=False)